In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time as t
import xlrd

In [2]:
baseline_params = pd.read_excel('BaselineParameters.xlsx', index_col=None, header=None)
a = baseline_params[0]
b = baseline_params[1]
mu = baseline_params.loc[0, 2]

In [3]:
filename='data/train.csv'
data=pd.read_csv(filename)

In [ ]:
len(data)

In [ ]:
n_u=len(data["userId"].unique())
n_m=len(data["movieId"].unique())


In [15]:
indices = np.random.permutation(len(data))
train_length = int(len(data)*0.8)
train_data = data.loc[indices[0:train_length]]
train_data = train_data.sort_values("userId")
validn_data = data.loc[indices[(train_length):]]
validn_data = validn_data.sort_values("userId")
train_data.reset_index(inplace=True,drop=True)
validn_data.reset_index(inplace=True,drop=True)

In [16]:
validn_data.shape

(1052868, 3)

In [10]:
total_5lakh=data.loc[indices[0:500000]]
train_4lakh = total_5lakh.loc[indices[0:400000]]

In [ ]:
#taking first 100000 entries 
data_new=data.head(100000)

In [5]:
def mse(actual,predicted):
    msqe=np.mean((actual-predicted)**2)
    return msqe
    

In [7]:
def sgd(eta,lamda,n_epochs,P,Q,data):
    predictions=np.zeros(len(data))
    train_err=[1e4]
    for epoch in range(n_epochs):
        t1=t.time()
        print("epoch no.",epoch)
        #going over all entries
        for k in range(len(data)):
            
            #extracting details from each row
            u=int(data.loc[k]['userId'])
            i=int(data.loc[k]['movieId'])
            
            
            #gradient descent
            e=data.loc[k]['rating']-(mu+a[u]+b[i]+np.dot(P[u,:],np.transpose(Q[i,:])))
            p_old=P[u,:]
            P[u,:] +=  eta*(e*Q[i,:]-lamda*P[u,:])
            Q[i,:] +=  eta*(e*p_old-lamda*Q[i,:])
            
            predictions[k]=mu + a[u] + b[i] + np.dot(P[u,:],np.transpose(Q[i,:]))
            
            if(k%10000==0):
                print(k,'training items done')
                
        t2=t.time()
        print("time for epoch no.",epoch,"=",t2-t1)

        train_err.append(mse(np.array(data['rating']),predictions))
        if(train_err[epoch+1]>train_err[epoch]):
            break
        if(np.isnan(np.sum(train_err))):
            break
        
        print("mean sq. err=",train_err[epoch+1])
        print('\n')
    return(mse(np.array(data['rating']),predictions),P,Q)


In [8]:
np.random.seed(1)
k=10 #no of features,to be adjusted
#feature matrix for users and movies
#first 100000 entries contain 185 users ,to be changed later
#10000 movies



begin=t.time()
predictions=np.zeros(len(train_data))
min_train_err=1e4
min_validn_err=1e4
tr_er_best_val=100
best_validn_param=[np.nan,np.nan]
for eta in [0.001,0.01,0.1]:
    for lamda in [0.01,0.1,1]:
        print("eta:",eta,"lambda",lamda)
        np.random.seed(1)
        P = 3*np.random.rand(10000,k)
        Q = 3*np.random.rand(10000,k)
       
        
        train_err,P,Q=sgd(eta,lamda,n_epochs=3,P=P,Q=Q,data=train_data)
        if(np.isnan(train_err)):
            break
            
        validn_err=validn_pred(P,Q)
        if(validn_err<min_validn_err):
            min_validn_err=validn_err
            best_validn_param=[eta,lamda]
            tr_er_best_val=train_err
       
print("mse valdn=",min_validn_err)   
print("mse train=",tr_er_best_val)
print("eta:",best_validn_param[0],"lambda:",best_validn_param[1])
        

end=t.time()
print("time taken = ",end-begin)

eta: 0.001 lambda 0.01
epoch no. 0
0 training items done
10000 training items done
20000 training items done
30000 training items done
40000 training items done
50000 training items done
60000 training items done
70000 training items done
80000 training items done
90000 training items done
100000 training items done
110000 training items done
120000 training items done
130000 training items done
140000 training items done
150000 training items done
160000 training items done
170000 training items done
180000 training items done
190000 training items done
200000 training items done
210000 training items done
220000 training items done
230000 training items done
240000 training items done
250000 training items done
260000 training items done
270000 training items done
280000 training items done
290000 training items done
300000 training items done
310000 training items done
320000 training items done
330000 training items done
340000 training items done
350000 training items done
360000 

2960000 training items done
2970000 training items done
2980000 training items done
2990000 training items done
3000000 training items done
3010000 training items done
3020000 training items done
3030000 training items done
3040000 training items done
3050000 training items done
3060000 training items done
3070000 training items done
3080000 training items done
3090000 training items done
3100000 training items done
3110000 training items done
3120000 training items done
3130000 training items done
3140000 training items done
3150000 training items done
3160000 training items done
3170000 training items done
3180000 training items done
3190000 training items done
3200000 training items done
3210000 training items done
3220000 training items done
3230000 training items done
3240000 training items done
3250000 training items done
3260000 training items done
3270000 training items done
3280000 training items done
3290000 training items done
3300000 training items done
3310000 training ite

1680000 training items done
1690000 training items done
1700000 training items done
1710000 training items done
1720000 training items done
1730000 training items done
1740000 training items done
1750000 training items done
1760000 training items done
1770000 training items done
1780000 training items done
1790000 training items done
1800000 training items done
1810000 training items done
1820000 training items done
1830000 training items done
1840000 training items done
1850000 training items done
1860000 training items done
1870000 training items done
1880000 training items done
1890000 training items done
1900000 training items done
1910000 training items done
1920000 training items done
1930000 training items done
1940000 training items done
1950000 training items done
1960000 training items done
1970000 training items done
1980000 training items done
1990000 training items done
2000000 training items done
2010000 training items done
2020000 training items done
2030000 training ite

380000 training items done
390000 training items done
400000 training items done
410000 training items done
420000 training items done
430000 training items done
440000 training items done
450000 training items done
460000 training items done
470000 training items done
480000 training items done
490000 training items done
500000 training items done
510000 training items done
520000 training items done
530000 training items done
540000 training items done
550000 training items done
560000 training items done
570000 training items done
580000 training items done
590000 training items done
600000 training items done
610000 training items done
620000 training items done
630000 training items done
640000 training items done
650000 training items done
660000 training items done
670000 training items done
680000 training items done
690000 training items done
700000 training items done
710000 training items done
720000 training items done
730000 training items done
740000 training items done
7

3330000 training items done
3340000 training items done
3350000 training items done
3360000 training items done
3370000 training items done
3380000 training items done
3390000 training items done
3400000 training items done
3410000 training items done
3420000 training items done
3430000 training items done
3440000 training items done
3450000 training items done
3460000 training items done
3470000 training items done
3480000 training items done
3490000 training items done
3500000 training items done
3510000 training items done
3520000 training items done
3530000 training items done
3540000 training items done
3550000 training items done
3560000 training items done
3570000 training items done
3580000 training items done
3590000 training items done
3600000 training items done
3610000 training items done
3620000 training items done
3630000 training items done
3640000 training items done
3650000 training items done
3660000 training items done
3670000 training items done
3680000 training ite

...

3800000 training items done
3810000 training items done
3820000 training items done
3830000 training items done
3840000 training items done
3850000 training items done
3860000 training items done
3870000 training items done
3880000 training items done
3890000 training items done
3900000 training items done
3910000 training items done
3920000 training items done
3930000 training items done
3940000 training items done
3950000 training items done
3960000 training items done
3970000 training items done
3980000 training items done
3990000 training items done
4000000 training items done
4010000 training items done
4020000 training items done
4030000 training items done
4040000 training items done
4050000 training items done
4060000 training items done
4070000 training items done
4080000 training items done
4090000 training items done
4100000 training items done
4110000 training items done
4120000 training items done
4130000 training items done
4140000 training items done
4150000 training ite

2530000 training items done
2540000 training items done
2550000 training items done
2560000 training items done
2570000 training items done
2580000 training items done
2590000 training items done
2600000 training items done
2610000 training items done
2620000 training items done
2630000 training items done
2640000 training items done
2650000 training items done
2660000 training items done
2670000 training items done
2680000 training items done
2690000 training items done
2700000 training items done
2710000 training items done
2720000 training items done
2730000 training items done
2740000 training items done
2750000 training items done
2760000 training items done
2770000 training items done
2780000 training items done
2790000 training items done
2800000 training items done
2810000 training items done
2820000 training items done
2830000 training items done
2840000 training items done
2850000 training items done
2860000 training items done
2870000 training items done
2880000 training ite

1250000 training items done
1260000 training items done
1270000 training items done
1280000 training items done
1290000 training items done
1300000 training items done
1310000 training items done
1320000 training items done
1330000 training items done
1340000 training items done
1350000 training items done
1360000 training items done
1370000 training items done
1380000 training items done
1390000 training items done
1400000 training items done
1410000 training items done
1420000 training items done
1430000 training items done
1440000 training items done
1450000 training items done
1460000 training items done
1470000 training items done
1480000 training items done
1490000 training items done
1500000 training items done
1510000 training items done
1520000 training items done
1530000 training items done
1540000 training items done
1550000 training items done
1560000 training items done
1570000 training items done
1580000 training items done
1590000 training items done
1600000 training ite

4180000 training items done
4190000 training items done
4200000 training items done
4210000 training items done
time for epoch no. 2 = 2530.4240469932556
0 validn items done
100000 validn items done
200000 validn items done
300000 validn items done
400000 validn items done
500000 validn items done
600000 validn items done
700000 validn items done
800000 validn items done
900000 validn items done
1000000 validn items done
eta: 0.1 lambda 0.01
epoch no. 0
0 training items done
10000 training items done
20000 training items done
30000 training items done
40000 training items done
50000 training items done
60000 training items done
70000 training items done
80000 training items done
90000 training items done
100000 training items done
110000 training items done
120000 training items done
130000 training items done
140000 training items done
150000 training items done
160000 training items done
170000 training items done
180000 training items done
190000 training items done
200000 training 

2820000 training items done
2830000 training items done
2840000 training items done
2850000 training items done
2860000 training items done
2870000 training items done
2880000 training items done
2890000 training items done
2900000 training items done
2910000 training items done
2920000 training items done
2930000 training items done
2940000 training items done
2950000 training items done
2960000 training items done
2970000 training items done
2980000 training items done
2990000 training items done
3000000 training items done
3010000 training items done
3020000 training items done
3030000 training items done
3040000 training items done
3050000 training items done
3060000 training items done
3070000 training items done
3080000 training items done
3090000 training items done
3100000 training items done
3110000 training items done
3120000 training items done
3130000 training items done
3140000 training items done
3150000 training items done
3160000 training items done
3170000 training ite

In [ ]:
plt.plot(train_err)

In [9]:
def validn_pred(P,Q):
    val_predn=[]
    for m in range(len(validn_data)):
            u=int(data.loc[m]['userId'])
            i=int(data.loc[m]['movieId'])
            val_predn.append(mu + a[u] + b[i] + np.dot(P[u,:],Q[i,:].T))
            if(m%100000==0):
                print(m,'validn items done')
    v_er=mse(np.array(validn_data['rating']),np.array(val_predn))
    print("validn error",v_er)
    return v_er
        
                         
        

In [ ]:
len(val_predn)

In [ ]:
print(mse(np.array(validn_data['rating']),np.array(val_predn)))

In [ ]:
val_predn[:10]

In [ ]:
validn_data['rating'][:10]

In [ ]:
filenametest='data/test.csv'
test_data=pd.read_csv(filenametest)

In [ ]:
len(test_data)

In [ ]:
test_predn=[]
for p in range(len(test_data)):
            
        test_predn.append(np.dot(P[int(test_data.loc[p]['userId']),:],Q[int(test_data.loc[p]['movieId']),:].T))
        
        if(p%10000==0):
            print(p,'test items done')

In [ ]:
test_predn_array=np.array(test_predn)

In [ ]:
ii=np.where(test_predn_array>5)

In [ ]:
test_predn_array[ii]=5

In [ ]:
test_predn_array=test_predn_array.round(1)

In [ ]:
id=range(len(test_data))

In [ ]:
dframe_list={"Id":id,"Prediction":test_predn_array}



test_dfrm=pd.DataFrame(dframe_list,columns=["Id","Prediction"])

test_dfrm.to_csv('latentmodel.csv',index=False)

In [ ]:
len(test_predn_array)

In [ ]:
ii=np.where(test_predn_array==5)
print(np.size(ii))